# PAC Learning Framework

In this section, I will concisely cover the Probably Approximately Correct (PAC) learning framework. 

## Motivation
In a sentence, the PAC framework quantifies the requirements (number of training points, time complexity, and space complexity) to approximately learn patterns in the data. THIS IS HUGE! For instance, a simple example is learning linear versus nonlinear maps between the input and outputs - linear patterns intuitively require less points to learn the pattern. Concretely, we could look at predicting height with weight data. If we desired a linear pattern, then we only need to search over $\mathbb{R}_+$ to find a parameter $\alpha$ such that $\text{height}=\alpha\cdot \text{weight}$. If instead we desire a nonlinear pattern, then think about the expansion of the space! We could fit polynomials, trigonometric, and the list goes on. I believe now intuition for patterns is more clear and we can turn to making our argument more rigorous and mathematical.

## Some Notations

- Let $\mathcal{X}$ denote the input space (includes all data excluding the output variable)

- Let $\mathcal{Y}$ denote the set of labels/output/target variables (as in [1], I limit discussion to binary outputs right now)

- Examples $(x,y)\in(\mathcal{X},\mathcal{Y})$ are assumed to be drawn independently and identically distributed (i.i.d.) according to some unknown distribution $\mathcal{D}$ 

- 

## Definitions

**Concept**: A concept $c:\mathcal{X}\rightarrow\mathcal{Y}$ is a mapping from $\mathcal{X}$ to $\mathcal{Y}$.

**Concept Class**: A concept class $\mathcal{C}$ is a set of concepts we may wish to learn. THESE 

## Examples

- Let us consider the features to be height, weight, age, ethnicity, and cancer (boolean) with the target being death (boolean). Then let us discuss concepts: for instance, all instances of Germans over 6' with weight >300lbs with cancer and over the age 55 get mapped to 1 i.e. they're dead (this is made up). This concept is thus an indicator function. We could consider a concept class to be a union of similar indicator variables. This type of concept class forms the basis for rule-based algorithms which will be discussed in a separate notebook.

- The book mentions rectangles or triangles as other alternatives for concept classes to go over. 

# Sources

[1] "Foundations of Machine Learning" by Mohri et al., 2012